In [3]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00


In [1]:
!pip install sentence-transformers

In [22]:
def retrieve_topics_from_namespaces(index, namespaces):
    """
    Retrieve topics from multiple namespaces and store them in separate lists.

    :param index: Pinecone Index object.
    :param namespaces: List of namespaces to query.
    :return: A tuple containing two lists: one for topics in the first namespace and one for topics in the second namespace.
    """
    topics_by_namespace = {}

    for namespace in namespaces:
        topics = set()  # Using set to avoid duplicate topics
        try:
            # Query with an empty vector to fetch all items in the namespace
            response = index.query(vector=[0] * 768, namespace=namespace, top_k=1000, include_metadata=True)
            # Extract topics from the metadata
            for match in response["matches"]:
                metadata = match.get("metadata", {})
                if "topic" in metadata:
                    topics.add(metadata["topic"])
        except Exception as e:
            print(f"Error retrieving metadata from namespace '{namespace}': {e}")

        # Store the topics for each namespace
        topics_by_namespace[namespace] = list(topics)  # Convert set to list

    return topics_by_namespace

# Example usage:
index_name = "mcq-index"
index = pc.Index(index_name)

# Specify the namespaces
namespaces = ["computer_organization", "operating_system"]

# Retrieve topics from both namespaces
topics_by_namespace = retrieve_topics_from_namespaces(index, namespaces)
print(topics_by_namespace)
# Print topics for each namespace
for namespace, topics in topics_by_namespace.items():
    print(f"Topics in namespace '{namespace}':")
    for topic in topics:
        print(f"- {topic}")


{'computer_organization': ['Computer Organization Architecture', 'Number Representation', 'Digital Logic Number Representation', 'Microprocessor'], 'operating_system': ['Input Output Systems', 'Memory Management', 'Cpu Scheduling', 'Dead Lock', 'Process Management', 'Unix']}
Topics in namespace 'computer_organization':
- Computer Organization Architecture
- Number Representation
- Digital Logic Number Representation
- Microprocessor
Topics in namespace 'operating_system':
- Input Output Systems
- Memory Management
- Cpu Scheduling
- Dead Lock
- Process Management
- Unix


In [23]:
import pinecone
from sentence_transformers import SentenceTransformer

# ... (your initialize_pinecone function, retrieve_topics_from_namespaces function and other setup) ...

def search_pinecone_by_topic(topic, namespace, top_k=10):
    """
    Searches Pinecone index for vectors similar to the given topic within a specific namespace.

    Args:
        topic: The topic to search for (string).
        namespace: The namespace to search within (string).
        top_k: The number of top results to return.

    Returns:
        A list of dictionaries, where each dictionary contains the ID, score and metadata of a match.
        Returns an empty list if there's an error or no matches are found.
    """
    try:
        model = SentenceTransformer('all-mpnet-base-v2')
        query_embedding = model.encode(topic)

        results = index.query(
            vector=query_embedding.tolist(),
            top_k=top_k,
            include_metadata=True,
            filter={"topic": {"$eq": topic}},  # Filter by topic
            namespace=namespace  # Specify the namespace
        )
        return results.matches

    except Exception as e:
        print(f"Error during Pinecone search: {e}")
        return []

# Example usage:
index_name = "mcq-index"
index = pc.Index(index_name)

# Specify the namespaces
namespaces = ["computer_organization", "operating_system"]

# Retrieve topics from both namespaces
topics_by_namespace = retrieve_topics_from_namespaces(index, namespaces)

# Search for a topic within a specific namespace
topic_to_search = "Number Representation"
namespace_to_search = "computer_organization"  # Choose the desired namespace

search_results = search_pinecone_by_topic(topic_to_search, namespace_to_search)

if search_results:
    for match in search_results:
        print(f"ID: {match.id}, Score: {match.score}, Metadata: {match.metadata}")
else:
    print("No results found.")

ID: Number Representation-Question25, Score: 0.406145602, Metadata: {'correct_option': 'C1CE0000', 'options': ['61CD0000', 'C1640000', 'C1CD0000', 'C1CE0000'], 'question_img_link': '', 'question_no': 'Question25', 'question_text': 'The value of a float type variable is represented using the single-precision 32-bit floating point format IEEE-754 standard that uses 1 bit for sign, 8 bits for biased exponent and 23 bits for mantissa. A float type variable X is assigned the decimal value of −25.75. The representation of X in hexadecimal notation is', 'topic': 'Number Representation'}
ID: Number Representation-Question7, Score: 0.404121786, Metadata: {'correct_option': 'C1640000H', 'options': ['C1640000H', '416C0000H', '41640000H', 'C16C0000H'], 'question_img_link': '', 'question_no': 'Question7', 'question_text': 'The value of a float type variable is represented using the single-precision 32-bit floating point format IEEE-754 standard that uses 1 bit for sign, 8 bits for biased exponent a

In [33]:

import pinecone
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


def retrieve_topics_from_namespaces(index, namespaces):
    """
    Retrieve topics from multiple namespaces and store them in separate lists.

    :param index: Pinecone Index object.
    :param namespaces: List of namespaces to query.
    :return: A tuple containing two lists: one for topics in the first namespace and one for topics in the second namespace.
    """
    topics_by_namespace = {}

    for namespace in namespaces:
        topics = set()  # Using set to avoid duplicate topics
        try:
            # Query with an empty vector to fetch all items in the namespace
            response = index.query(vector=[0] * 768, namespace=namespace, top_k=1000, include_metadata=True)
            # Extract topics from the metadata
            for match in response["matches"]:
                metadata = match.get("metadata", {})
                if "topic" in metadata:
                    topics.add(metadata["topic"])
        except Exception as e:
            print(f"Error retrieving metadata from namespace '{namespace}': {e}")

        # Store the topics for each namespace
        topics_by_namespace[namespace] = list(topics)  # Convert set to list

    return topics_by_namespace


# Initialize the embedding model
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")


# Function to extract the most relevant topic (replace with your implementation)
def extract_topic_from_query(query, topics, model):
    # Replace this with your actual implementation to extract the topic
    # This is a placeholder, remove it and add your logic here
    # ... (Your implementation to extract the topic) ...
    # For now, returning the first topic as a placeholder
    # You should use your model and logic here
    if topics:
        return topics[0]
    else:
        return None


# Function to retrieve MCQs (modified to return JSON)
def get_mcqs_by_topic(index, topic, namespaces, question_range=None):
    mcqs = []
    for namespace in namespaces:  # Iterate through each namespace
        try:
            filter_conditions = {"topic": topic}
            query_response = index.query(
                vector=[0] * 768,
                filter=filter_conditions,
                top_k=100,
                namespace=namespace  # Pass a single namespace here
            )

            # Extract MCQs and format as JSON
            for match in query_response.matches:
                metadata = match.metadata
                mcq_data = {
                    "id": match.id,
                    "topic": metadata.get("topic"),
                    "question": metadata.get("question"),
                    # ... add other metadata fields you want to include
                }
                mcqs.append(mcq_data)

        except Exception as e:
            print(f"Error retrieving MCQs from namespace '{namespace}': {e}")

    return json.dumps(mcqs, indent=2)  # Return JSON string


def search_pinecone_by_topic(query, namespaces, top_k=10):
    """
    Searches Pinecone index for vectors similar to the given topic within specified namespaces.

    Args:
        query: The user's query (string).
        namespaces: The namespaces to search within (list of strings).
        top_k: The number of top results to return.

    Returns:
        A JSON string containing the retrieved MCQs.
    """
    try:
        # Get all topics from the specified namespaces
        topics_by_namespace = retrieve_topics_from_namespaces(index, namespaces)
        all_topics = [topic for topics in topics_by_namespace.values() for topic in topics]

        # Extract the most relevant topic based on the query
        extracted_topic = extract_topic_from_query(query, all_topics, embedding_model)

        # Retrieve MCQs for the extracted topic
        mcqs_json = get_mcqs_by_topic(index, extracted_topic, namespaces)

        return mcqs_json

    except Exception as e:
        print(f"Error during Pinecone search: {e}")
        return json.dumps({"error": str(e)})


# Example usage:
# Make sure to replace these with your actual Pinecone API key and environment
pc = Pinecone(api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4", environment="us-west1-gcp")

index_name = "mcq-index"
index = pc.Index(index_name)
namespaces = ["computer_organization", "operating_system"]
user_query = "Tell me about microprocessors"

search_results_json = search_pinecone_by_topic(user_query, namespaces)
print(search_results_json)

Error retrieving MCQs from namespace 'computer_organization': 'NoneType' object has no attribute 'get'
[]


In [34]:
import pinecone
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize Pinecone
# Replace these with your actual Pinecone API key and environment
pc = Pinecone(api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4", environment="us-west1-gcp")
index_name = "mcq-index"
index = pc.Index(index_name)

# Initialize the embedding model
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

def retrieve_topics_from_namespaces(index, namespaces):
    """
    Retrieve topics from multiple namespaces and store them in separate lists.
    """
    topics_by_namespace = {}

    for namespace in namespaces:
        topics = set()  # Using set to avoid duplicate topics
        try:
            response = index.query(vector=[0] * 768, namespace=namespace, top_k=1000, include_metadata=True)
            for match in response["matches"]:
                metadata = match.get("metadata", {})
                if "topic" in metadata:
                    topics.add(metadata["topic"])
        except Exception as e:
            print(f"Error retrieving metadata from namespace '{namespace}': {e}")

        topics_by_namespace[namespace] = list(topics)  # Convert set to list

    return topics_by_namespace

def extract_topic_from_query(query, topics, model):
    """
    Extract the most relevant topic based on the query.

    (Placeholder - Replace with your actual topic extraction logic)
    """
    # ... (Your implementation to extract the topic) ...
    if topics:
        return topics[0]  # Returning the first topic as a placeholder
    else:
        return None

def search_pinecone_by_topic(query, namespaces, top_k=10):
    """
    Searches Pinecone index for vectors similar to the given query within specified namespaces.
    """
    try:
        model = SentenceTransformer('all-mpnet-base-v2')
        query_embedding = model.encode(query)

        # Get all topics from specified namespaces
        topics_by_namespace = retrieve_topics_from_namespaces(index, namespaces)
        all_topics = [topic for topics in topics_by_namespace.values() for topic in topics]

        # Extract relevant topic based on the query
        extracted_topic = extract_topic_from_query(query, all_topics, embedding_model)

        # Search across specified namespaces
        mcqs = []
        for namespace in namespaces:
            results = index.query(
                vector=query_embedding.tolist(),
                top_k=top_k,
                include_metadata=True,
                filter={"topic": {"$eq": extracted_topic}},  # Filter by topic
                namespace=namespace
            )

            # Process results and filter for valid metadata
            for match in results.matches:
                metadata = match.metadata
                if metadata and metadata.get("question_text"):  # Check for valid metadata
                    mcqs.append(metadata)

        return json.dumps(mcqs, indent=2)

    except Exception as e:
        print(f"Error during Pinecone search: {e}")
        return json.dumps({"error": str(e)})

# Example usage:
namespaces = ["computer_organization", "operating_system"]
user_query = "Tell me about microprocessors"

search_results_json = search_pinecone_by_topic(user_query, namespaces)
print(search_results_json)

[
  {
    "correct_option": "a, b, c and d",
    "options": [
      "Only a",
      "a and b",
      "c and d",
      "a, b, c and d"
    ],
    "question_img_link": "",
    "question_no": "Question117",
    "question_text": "A certain processor supports only the immediate and the direct addressing modes. Which of the following programming language features cannot be implemented on this processor?\r\na) Pointers\r\nb) Arrays\r\nc) Records\r\nd) Recursive procedures with local variables",
    "topic": "Computer Organization Architecture"
  },
  {
    "correct_option": "31",
    "options": [
      "16",
      "31",
      "32",
      "None"
    ],
    "question_img_link": "",
    "question_no": "Question90",
    "question_text": "A processor can support a maximum memory of 4 GB, where the memory is word-addressable (a  word  consists  of  two  bytes).   The  size  of  the  address  bus  of  the  processor  is  at ____  least bits.\r\n\u00a0\r\nNote : This question was asked as Numerical A